<a href="https://colab.research.google.com/github/ykamen/CS4342/blob/main/hw1part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!python --version
import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from google.colab import drive

Python 3.7.12


In [17]:
def fPC (y, yhat):
    A = confusion_matrix(y,yhat)
    pc = (A[0][0] + A[1][1])/len(y)
    return pc

In [ ]:
drive.mount('/content/drive')

In [27]:
faces = np.load('drive/MyDrive/trainingFaces.npy', 'r')

In [21]:
y = [0,0,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1]
yhat = [1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1]
fPC(y,yhat)

0.9552238805970149

In [41]:
print(faces[0][575])

-0.6677621653616024


One predictors comparison to all other pixels

In [119]:
def p13(x,k):
  return np.tile(x,(k,1))

def jcomp(arr,p1):
  vector = arr[:,p1]
  vmaptemp = p13(vector,arr.shape[1])
  vmap = vmaptemp.transpose()
  return vmap > arr

In [120]:
jcomp(faces,2)

array([[False, False, False, ...,  True,  True, False],
       [False, False, False, ...,  True, False,  True],
       [ True,  True, False, ..., False, False, False],
       ...,
       [ True,  True, False, ...,  True, False,  True],
       [ True, False, False, ...,  True,  True,  True],
       [ True, False, False, ...,  True,  True,  True]])

In [114]:
print(faces)

[[-0.27874693 -0.61692736 -0.86067416 ... -1.07838038 -0.91595662
  -0.66776217]
 [-0.63853749 -0.48942235 -0.63860372 ... -0.75003974 -0.35491409
  -1.32539541]
 [-0.68737551 -0.74619822  0.9053906  ...  1.0791425   1.99718399
   2.04014052]
 ...
 [-0.99305176 -0.86931048 -0.05987676 ... -0.61200392  0.84946658
  -0.14246518]
 [ 0.7667242   1.14034143  1.09812381 ... -1.70468228 -1.10556106
  -0.67715482]
 [-0.7654332   0.88344899  0.53067434 ... -0.42550697 -0.56036467
  -0.19055345]]
